In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader, WeightedRandomSampler

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score

from f2_preprocessor import Preprocessor

import optuna

from tabm import TabM
from rtdl_num_embeddings import LinearReLUEmbeddings  # simple but good :contentReference[oaicite:1]{index=1}

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
num_cols = [
    "year_of_born",
    "email_or_tel_available",
    "safety_rating",
    "annual_income",
    "high_education_ind",
    "address_change_ind",
    "past_num_of_claims",
    "liab_prct",
    "policy_report_filed_ind",
    "claim_est_payout",
    "vehicle_made_year",
    "vehicle_price",
    "vehicle_weight",
    "age_of_DL",
    "vehicle_mileage",
]

cat_cols = [
    "gender",
    "living_status",
    "zip_code",
    "claim_day_of_week",
    "accident_site",
    "witness_present_ind",
    "channel",
    "vehicle_category",
    "vehicle_color",
    "accident_type",
    "in_network_bodyshop",
]

In [3]:
def compute_best_f1(probs: np.ndarray,
                    targets: np.ndarray,
                    thresholds: np.ndarray | None = None):
    """
    probs:    shape (N,), predicted positive probabilities
    targets:  shape (N,), 0/1 labels
    thresholds: optional array of candidate thresholds (0..1)

    Returns (best_f1, best_threshold)
    """
    if thresholds is None:
        # Coarse but reasonable grid; you can make it denser if you want
        thresholds = np.linspace(0.05, 0.95, 19)

    best_f1 = 0.0
    best_t = 0.5

    for t in thresholds:
        preds = (probs >= t).astype(int)
        f1 = f1_score(targets, preds, zero_division=0)
        if f1 > best_f1:
            best_f1 = f1
            best_t = t

    return best_f1, best_t


In [4]:
df = pd.read_csv("data/Training_TriGuard.csv")
df = df.dropna(subset=['subrogation'])

df = df.drop(columns=["claim_number"], errors="ignore")

target_col = "subrogation"
y_all = df[target_col].to_numpy().astype(np.int64)

In [5]:
# Compute global pos_weight for BCEWithLogitsLoss
pos_frac = y_all.mean()
neg_frac = 1.0 - pos_frac

# Avoid division by zero – but you almost surely have both classes
pos_weight_value = neg_frac / max(pos_frac, 1e-6)
print("Positive class fraction:", pos_frac)
print("pos_weight:", pos_weight_value)

device = "cpu"

Positive class fraction: 0.2286238124340241
pos_weight: 3.373997569866343


In [6]:
def build_model(n_num_features: int,
                cat_cardinalities,
                trial: optuna.Trial) -> nn.Module:
    # EDIT: tighten but beef up architecture search space
    n_blocks = trial.suggest_int("n_blocks", 3, 5)                 # was 2–5
    d_block  = trial.suggest_int("d_block", 384, 1024, log=True)   # was 256–1024
    dropout  = trial.suggest_float("dropout", 0.0, 0.4)
    k        = trial.suggest_int("k", 8, 32, step=8)               # keep 8–32

    num_emb = LinearReLUEmbeddings(n_num_features)

    model = TabM.make(
        n_num_features=n_num_features,
        num_embeddings=num_emb,
        cat_cardinalities=cat_cardinalities,
        d_out=1,
        n_blocks=n_blocks,
        d_block=d_block,
        dropout=dropout,
        k=k,
    )

    return model.to(device)

In [7]:
def train_one_fold(model,
                   train_loader,
                   valid_loader,
                   n_epochs: int,
                   lr: float,
                   weight_decay: float,
                   pos_weight_eff: float) -> float:
    # EDIT: pass in effective pos_weight (after scaling)
    pos_weight = torch.tensor([pos_weight_eff], device=device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=lr,
        weight_decay=weight_decay,
    )

    # -------- TRAIN --------
    for epoch in range(n_epochs):
        model.train()
        for X_num_b, X_cat_b, y_b in train_loader:
            X_num_b = X_num_b.to(device)
            X_cat_b = X_cat_b.to(device)
            y_b = y_b.to(device)   # (B, 1)

            logits = model(X_num_b, X_cat_b)          # (B, k, 1)
            B, k, _ = logits.shape

            # EDIT: use reshape instead of view
            logits_flat = logits.reshape(B * k, 1)    # (B*k, 1)
            y_flat = y_b.repeat_interleave(k, dim=0)  # (B*k, 1)

            loss = criterion(logits_flat, y_flat)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # -------- VALID: compute probs and F1 --------
    model.eval()
    all_probs = []
    all_targets = []

    with torch.no_grad():
        for X_num_b, X_cat_b, y_b in valid_loader:
            X_num_b = X_num_b.to(device)
            X_cat_b = X_cat_b.to(device)
            y_b = y_b.to(device)

            logits = model(X_num_b, X_cat_b)              # (B, k, 1)
            probs = torch.sigmoid(logits).mean(dim=1)     # (B, 1)
            probs = probs.squeeze(-1).cpu().numpy()
            targets = y_b.squeeze(-1).cpu().numpy()

            all_probs.append(probs)
            all_targets.append(targets)

    all_probs = np.concatenate(all_probs)
    all_targets = np.concatenate(all_targets)

    # Optional defensive mask
    mask = ~np.isnan(all_probs) & ~np.isnan(all_targets)
    all_probs = all_probs[mask]
    all_targets = all_targets[mask]

    if len(np.unique(all_targets)) < 2:
        return 0.0

    best_f1, best_t = compute_best_f1(all_probs, all_targets)
    return float(best_f1)


In [12]:
def objective(trial: optuna.Trial) -> float:
    # EDIT: more suitable ranges for TabM on imbalanced data
    lr = trial.suggest_float("lr", 5e-4, 1e-2, log=True)                 # was 1e-4–3e-3
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [64, 128, 256]) # was 256–1024
    n_epochs = trial.suggest_int("n_epochs", 20, 40)                     # was 5–10

    # EDIT: let Optuna scale pos_weight
    pos_weight_scale = trial.suggest_float("pos_weight_scale", 0.5, 3.0)
    pos_weight_eff = pos_weight_value * pos_weight_scale

    n_splits = 3
    skf = StratifiedKFold(
        n_splits=n_splits,
        shuffle=True,
        random_state=42,
    )

    fold_f1s = []

    for fold_idx, (train_idx, valid_idx) in enumerate(skf.split(df, y_all)):
        train_df = df.iloc[train_idx].reset_index(drop=True)
        valid_df = df.iloc[valid_idx].reset_index(drop=True)

        preproc = Preprocessor(num_cols=num_cols, cat_cols=cat_cols,
                                   target_col=target_col)

        X_num_train, X_cat_train, y_train = preproc.fit_transform(train_df)
        X_num_valid, X_cat_valid, y_valid = preproc.transform(valid_df)

        n_num_features = X_num_train.shape[1]
        cat_cardinalities = [
            preproc.cat_cardinalities_[c] for c in cat_cols
        ]

        model = build_model(n_num_features, cat_cardinalities, trial)

        X_num_tr_t = torch.from_numpy(X_num_train)
        X_cat_tr_t = torch.from_numpy(X_cat_train)
        y_tr_t = torch.from_numpy(y_train)

        X_num_va_t = torch.from_numpy(X_num_valid)
        X_cat_va_t = torch.from_numpy(X_cat_valid)
        y_va_t = torch.from_numpy(y_valid)

        train_dataset = TensorDataset(X_num_tr_t, X_cat_tr_t, y_tr_t)
        valid_dataset = TensorDataset(X_num_va_t, X_cat_va_t, y_va_t)

        # Imbalance-aware sampler
        y_train_flat = y_train.ravel()
        class_counts = np.bincount(y_train_flat.astype(int))
        class_counts = np.maximum(class_counts, 1)
        class_weights = 1.0 / class_counts
        sample_weights = class_weights[y_train_flat.astype(int)]

        sampler = WeightedRandomSampler(
            weights=torch.from_numpy(sample_weights).float(),
            num_samples=len(sample_weights),
            replacement=True,
        )

        train_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            sampler=sampler,
            drop_last=False,
        )
        valid_loader = DataLoader(
            valid_dataset,
            batch_size=batch_size,
            shuffle=False,
            drop_last=False,
        )

        fold_f1 = train_one_fold(
            model=model,
            train_loader=train_loader,
            valid_loader=valid_loader,
            n_epochs=n_epochs,
            lr=lr,
            weight_decay=weight_decay,
            pos_weight_eff=pos_weight_eff,       # EDIT: pass scaled pos_weight
        )
        fold_f1s.append(fold_f1)

        trial.report(fold_f1, step=fold_idx)
        if trial.should_prune():
            raise optuna.TrialPruned()

    mean_f1 = float(np.mean(fold_f1s))
    return mean_f1


In [13]:
pruner = optuna.pruners.MedianPruner(n_warmup_steps=1)

study = optuna.create_study(
    direction="maximize",
    study_name="tabm_subrogation_cv_f1",
    pruner=pruner
)
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("Best mean CV F1:", study.best_value)
print("Best params:", study.best_params)


[I 2025-11-13 06:31:37,618] A new study created in memory with name: tabm_subrogation_cv_f1
Best trial: 0. Best value: 0.447372:   2%|▏         | 1/50 [11:00<8:59:04, 660.10s/it]

[I 2025-11-13 06:42:37,718] Trial 0 finished with value: 0.44737241605974015 and parameters: {'lr': 0.0006894437249902676, 'weight_decay': 0.0075725263336484565, 'batch_size': 256, 'n_epochs': 22, 'pos_weight_scale': 1.2057968350977348, 'n_blocks': 4, 'd_block': 803, 'dropout': 0.3888584189356692, 'k': 32}. Best is trial 0 with value: 0.44737241605974015.


Best trial: 0. Best value: 0.447372:   4%|▍         | 2/50 [14:14<5:08:52, 386.10s/it]

[I 2025-11-13 06:45:52,020] Trial 1 finished with value: 0.37216242076746936 and parameters: {'lr': 0.0008640415443663024, 'weight_decay': 0.0007194560755780915, 'batch_size': 128, 'n_epochs': 33, 'pos_weight_scale': 1.6997170578629168, 'n_blocks': 3, 'd_block': 707, 'dropout': 0.38597094914730634, 'k': 8}. Best is trial 0 with value: 0.44737241605974015.


Best trial: 2. Best value: 0.514259:   6%|▌         | 3/50 [17:55<4:03:34, 310.94s/it]

...

[I 2025-11-13 11:56:02,985] Trial 47 pruned. 


Best trial: 40. Best value: 0.59381:  98%|█████████▊| 49/50 [5:29:11<05:48, 348.05s/it]

[I 2025-11-13 12:00:49,548] Trial 48 finished with value: 0.5912304099386172 and parameters: {'lr': 0.0014896832680515405, 'weight_decay': 3.318081098209326e-06, 'batch_size': 64, 'n_epochs': 32, 'pos_weight_scale': 1.3741210222490037, 'n_blocks': 5, 'd_block': 611, 'dropout': 0.1439113240821146, 'k': 8}. Best is trial 40 with value: 0.5938096041958789.


Best trial: 40. Best value: 0.59381: 100%|██████████| 50/50 [5:34:31<00:00, 401.43s/it]

[I 2025-11-13 12:06:08,994] Trial 49 pruned. 
Best mean CV F1: 0.5938096041958789
Best params: {'lr': 0.0011183425690718294, 'weight_decay': 0.004187499814600279, 'batch_size': 64, 'n_epochs': 33, 'pos_weight_scale': 2.159985493642329, 'n_blocks': 4, 'd_block': 455, 'dropout': 0.18355459979095332, 'k': 24}


In [25]:
best_params = study.best_params
print(best_params)

{'lr': 0.0011183425690718294, 'weight_decay': 0.004187499814600279, 'batch_size': 64, 'n_epochs': 33, 'pos_weight_scale': 2.159985493642329, 'n_blocks': 4, 'd_block': 455, 'dropout': 0.18355459979095332, 'k': 24}


In [26]:
# Refit preprocessor on the full training data
preproc_final = Preprocessor(num_cols=num_cols, cat_cols=cat_cols, target_col=target_col)
X_num_full, X_cat_full, y_full = preproc_final.fit_transform(df)

# Prepare tensors
X_num_full_t = torch.from_numpy(X_num_full)
X_cat_full_t = torch.from_numpy(X_cat_full)
y_full_t     = torch.from_numpy(y_full)

full_dataset = TensorDataset(X_num_full_t, X_cat_full_t, y_full_t)

full_loader = DataLoader(
    full_dataset,
    batch_size=best_params["batch_size"],
    shuffle=True,
    drop_last=False
)

In [27]:
# Build final TabM model from best params
model_final = build_model(
    n_num_features=X_num_full.shape[1],
    cat_cardinalities=[preproc_final.cat_cardinalities_[c] for c in cat_cols],
    trial=optuna.trial.FixedTrial(best_params)
)

In [28]:
# No weight, handle imbalance through threshold optimizing
criterion = nn.BCEWithLogitsLoss()

optimizer = torch.optim.AdamW(
    model_final.parameters(),
    lr=best_params["lr"],
    weight_decay=best_params["weight_decay"],
)

n_epochs = best_params["n_epochs"]

for epoch in range(n_epochs):
    model_final.train()
    running_loss = 0.0
    for Xn, Xc, yb in full_loader:
        Xn = Xn.to(device)
        Xc = Xc.to(device)
        yb = yb.to(device)

        logits = model_final(Xn, Xc)        # (B, k, 1)
        B, k, _ = logits.shape
        logits_flat = logits.reshape(B * k, 1)
        y_flat = yb.repeat_interleave(k, dim=0)

        loss = criterion(logits_flat, y_flat)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    if (epoch+1) % 5 == 0:
        print(f"Epoch {epoch+1}/{n_epochs}, loss={running_loss:.4f}")


Epoch 5/33, loss=140.5011
Epoch 10/33, loss=121.5093
Epoch 15/33, loss=113.7620
Epoch 20/33, loss=112.7051
Epoch 25/33, loss=111.7766
Epoch 30/33, loss=111.8216


In [37]:
# Diagnostic Code

from sklearn.metrics import f1_score, confusion_matrix

model_final.eval()
with torch.no_grad():
    Xn_tr = X_num_full_t.to(device)
    Xc_tr = X_cat_full_t.to(device)
    logits_tr = model_final(Xn_tr, Xc_tr)                 # (N, k, 1)
    probs_tr = torch.sigmoid(logits_tr).mean(dim=1)       # (N, 1)
    probs_tr = probs_tr.squeeze(-1).cpu().numpy()

print("Train probs min/mean/max:", probs_tr.min(), probs_tr.mean(), probs_tr.max())

y_train_true = y_full.ravel().astype(int)
y_train_pred_03 = (probs_tr >= 0.3).astype(int)

print("Train F1 @0.3:", f1_score(y_train_true, y_train_pred_03))
print("Train confusion @0.5:\n", confusion_matrix(y_train_true, y_train_pred_05))


Train probs min/mean/max: 0.0005119745 0.25603554 0.842113
Train F1 @0.3: 0.5988591317799478
Train confusion @0.5:
 [[12912   972]
 [ 2268  1847]]


In [31]:
best_f1_train, best_t_train = compute_best_f1(probs_tr, y_full.ravel().astype(int))
print("Best F1 on train:", best_f1_train)
print("Best threshold:", best_t_train)

Best F1 on train: 0.5988591317799478
Best threshold: 0.3


In [32]:
real_test = pd.read_csv("data/Testing_TriGuard.csv")

# Keep columns needed for domain features
claim_numbers = real_test["claim_number"].copy()

# Use the final preprocessor (fit on full training data!)
X_num_test, X_cat_test, _ = preproc_final.transform(real_test)

X_num_test_t = torch.from_numpy(X_num_test)
X_cat_test_t = torch.from_numpy(X_cat_test)


In [33]:
model_final.eval()
all_probs = []

with torch.no_grad():
    Xn = X_num_test_t.to(device)
    Xc = X_cat_test_t.to(device)

    logits = model_final(Xn, Xc)                 # (N, k, 1)
    probs = torch.sigmoid(logits).mean(dim=1)    # (N, 1)
    probs = probs.squeeze(-1).cpu().numpy()

    all_probs = probs


In [34]:
real_pred_proba = all_probs
real_pred_label = (real_pred_proba >= best_t_train).astype(int)

In [35]:
# Diagnostic Code

print("Min prob:", real_pred_proba.min())
print("Max prob:", real_pred_proba.max())
print("Mean prob:", real_pred_proba.mean())
print("First 20 probs:", real_pred_proba[:20])
print("First 20 labels:", real_pred_label[:20])
print("Unique labels:", np.unique(real_pred_label))


Min prob: 0.0007758692
Max prob: 0.82720715
Mean prob: 0.25367633
First 20 probs: [0.21576667 0.3347236  0.06319809 0.609448   0.36384344 0.03800942
 0.45161465 0.06597612 0.23697914 0.66205806 0.09418901 0.47217855
 0.2660694  0.2500148  0.22387505 0.58648896 0.054098   0.04736322
 0.15361674 0.4636856 ]
First 20 labels: [0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 1 0 0 0 1]
Unique labels: [0 1]


In [36]:
prediction = pd.DataFrame({
    "claim_number": claim_numbers,
    "subrogation": real_pred_label
})

prediction.to_csv("results/tabm_5938_prediction.csv", index=False)

print("Saved:", "results/tabm_5938_prediction.csv")


Saved: results/tabm_5938_prediction.csv


Model Saving Pipeline

In [38]:
from TabM_save_load import TabM_save_load

In [40]:
# 1. Create the pipeline wrapper
pipeline = TabM_save_load(
    model=model_final,
    preprocessor=preproc_final,
    threshold=best_t_train,
    best_params=best_params,
    num_cols=num_cols,
    cat_cols=cat_cols,
    device=device
)

# 2. Save everything to a folder
save_dir = "models/tabm_full_pipeline_5938"
pipeline.save(save_dir)

TabM pipeline saved successfully to models/tabm_full_pipeline_5938
